# Adjust GLiNER model with contrastive learning (NCELoss)

In [ ]:
!pip install gliner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.3/76.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 89.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 93.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 91.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.0 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.0
    Uninstalling tokenizers-0.22.0:
      Successfully uninstalled tokenizers-0.22.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.1
    Uninstalling transformers-4.56.1:
      Successfully uninstalled transformers-4.56.1


In [ ]:
!pip install transformers

In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
import json

import torch

from gliner.training import Trainer, TrainingArguments
from gliner.data_processing.collator import DataCollator
from gliner import GLiNER

In [ ]:
# Load the processed output from the JSON file which contains data ready for training
with open("processed_output.json", "r", encoding="utf-8") as file:
    processed_output = json.load(file)

print(len(processed_output))
print(processed_output[0])

2995
{'tokenized_text': ['Any', 'part', 'of', 'your', 'neck', 'muscles', ',', 'bones', ',', 'joints', ',', 'tendons', ',', 'ligaments', ',', 'or', 'nerves', 'can', 'cause', 'neck', 'problems', '.', 'Neck', 'pain', 'is', 'very', 'common', '.', 'Pain', 'may', 'also', 'come', 'from', 'your', 'shoulder', ',', 'jaw', ',', 'head', ',', 'or', 'upper', 'arms', '.', 'Muscle', 'strain', 'or', 'tension', 'often', 'causes', 'neck', 'pain', '.', 'The', 'problem', 'is', 'usually', 'overuse', ',', 'such', 'as', 'from', 'sitting', 'at', 'a', 'computer', 'for', 'too', 'long', '.', 'Sometimes', 'you', 'can', 'strain', 'your', 'neck', 'muscles', 'from', 'sleeping', 'in', 'an', 'awkward', 'position', 'or', 'overdoing', 'it', 'during', 'exercise', '.', 'Falls', 'or', 'accidents', ',', 'including', 'car', 'accidents', ',', 'are', 'another', 'common', 'cause', 'of', 'neck', 'pain', '.', 'Whiplash', ',', 'a', 'soft', 'tissue', 'injury', 'to', 'the', 'neck', ',', 'is', 'also', 'called', 'neck', 'sprain', 'or',

In [ ]:
train_dataset = processed_output[:int(len(processed_output)*0.9)]
test_dataset = processed_output[int(len(processed_output)*0.9):]

In [ ]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
gliner_model_base_ft = GLiNER.from_pretrained("urchade/gliner_small")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

gliner_config.json:   0%|          | 0.00/477 [00:00<?, ?B/s]

.gitattributes: 0.00B [00:00, ?B/s]

README.md: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/611M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
gliner_model_base_ft.config

GLiNERConfig {
  "class_token_index": 128002,
  "decoder_mode": null,
  "dropout": 0.4,
  "embed_ent_token": true,
  "encoder_config": {
    "_attn_implementation_autoset": true,
    "_name_or_path": "microsoft/deberta-v3-small",
    "add_cross_attention": false,
    "architectures": null,
    "attention_probs_dropout_prob": 0.1,
    "bad_words_ids": null,
    "begin_suppress_tokens": null,
    "bos_token_id": null,
    "chunk_size_feed_forward": 0,
    "cross_attention_hidden_size": null,
    "decoder_start_token_id": null,
    "diversity_penalty": 0.0,
    "do_sample": false,
    "early_stopping": false,
    "encoder_no_repeat_ngram_size": 0,
    "eos_token_id": null,
    "exponential_decay_length_penalty": null,
    "finetuning_task": null,
    "forced_bos_token_id": null,
    "forced_eos_token_id": null,
    "hidden_act": "gelu",
    "hidden_dropout_prob": 0.1,
    "hidden_size": 768,
    "id2label": {
      "0": "LABEL_0",
      "1": "LABEL_1"
    },
    "initializer_range": 0.02,

In [ ]:
gliner_model_base_ft.to(device)

GLiNER(
  (model): SpanModel(
    (token_rep_layer): Encoder(
      (bert_layer): Transformer(
        (model): DebertaV2Model(
          (embeddings): DebertaV2Embeddings(
            (word_embeddings): Embedding(128004, 768, padding_idx=0)
            (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (encoder): DebertaV2Encoder(
            (layer): ModuleList(
              (0-5): 6 x DebertaV2Layer(
                (attention): DebertaV2Attention(
                  (self): DisentangledSelfAttention(
                    (query_proj): Linear(in_features=768, out_features=768, bias=True)
                    (key_proj): Linear(in_features=768, out_features=768, bias=True)
                    (value_proj): Linear(in_features=768, out_features=768, bias=True)
                    (pos_dropout): Dropout(p=0.1, inplace=False)
                    (dropout): Dropout(p=0.1, inplace=False)
              

In [ ]:
data_collator_base_ft = DataCollator(gliner_model_base_ft.config, data_processor=gliner_model_base_ft.data_processor, prepare_labels=True)

In [ ]:
num_steps = 5300
batch_size = 8
data_size = len(train_dataset) # 2700
num_batches = data_size // batch_size
num_epochs = max(1, num_steps // num_batches)

In [ ]:
num_epochs

15

In [ ]:
training_args_base_ft = TrainingArguments(
    output_dir="gliner_model_base_ft",
    learning_rate=5e-3,
    weight_decay=0.01,
    lr_scheduler_type="linear",
    warmup_ratio=0.1,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    dataloader_num_workers=0,
    use_cpu=False,
    report_to="none",
    save_strategy="steps",
    eval_strategy="steps",
    eval_steps=338,
    save_steps=338,
    load_best_model_at_end=False,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    save_total_limit=3,
    fp16 = True
)

In [ ]:
import torch
from transformers import TrainerCallback

class ClearCacheCallback(TrainerCallback):
    def on_step_end(self, args, state, control, **kwargs):
        torch.cuda.empty_cache()

In [ ]:
trainer_base_ft = Trainer(
    model=gliner_model_base_ft,
    args=training_args_base_ft,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=gliner_model_base_ft.data_processor.transformer_tokenizer,
    data_collator=data_collator_base_ft,
    callbacks=[ClearCacheCallback()]
)

/tmp/ipython-input-373685390.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_base_ft = Trainer(


In [ ]:
print("Allocated memory:", torch.cuda.memory_allocated() / 1024**3, "GB")

Allocated memory: 0.56866455078125 GB


In [ ]:
torch.cuda.empty_cache()

In [ ]:
trainer_base_ft.train()

/usr/local/lib/python3.12/dist-packages/gliner/data_processing/processor.py:351: UserWarning: Sentence of length 424 has been truncated to 384
  warnings.warn(f"Sentence of length {len(tokens)} has been truncated to {max_len}")
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/usr/local/lib/python3.12/dist-packages/gliner/data_processing/processor.py:351: UserWarning: Sentence of length 1018 has been truncated to 384
  warnings.warn(f"Sentence of length {len(tokens)} has been truncated to {max_len}")


Step,Training Loss,Validation Loss
338,No log,191818.031250
676,194360.448000,196770.046875
1014,135882.592000,192007.484375
1352,135882.592000,194201.921875
1690,131483.480000,199303.765625
2028,132683.464000,198763.437500
2366,132683.464000,208876.078125
2704,132718.480000,198014.906250
3042,133654.072000,208512.140625
3380,133654.072000,197705.046875


Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.12/dist-packages/gliner/data_processing/processor.py:351: UserWarning: Sentence of length 578 has been truncated to 384
  warnings.warn(f"Sentence of length {len(tokens)} has been truncated to {max_len}")
/usr/local/lib/python3.12/dist-packages/gliner/data_processing/processor.py:351: UserWarning: Sentence of length 513 has been truncated to 384
  warnings.warn(f"Sentence of length {len(tokens)} has been truncated to {max_len}")
/usr/local/lib/python3.12/dist-packages/gliner/data_processing/processor.py:351: UserWarning: Sentence of length 1004 has been truncated to 384
  warnings.warn(f"Sentence of length {len(tokens)} has been truncated to {max_len}")
/usr/local/lib/python3.12/dist-packages/gliner/data_processing/processor.py:351: UserWarning: Sentence of length 641 has been truncated to 384
  warnings.warn(f"Sentence of length {len(tokens)} has been truncated to {max_len}")
/usr/local/lib/python3.12/dist-packa

TrainOutput(global_step=5055, training_loss=139655.40731948565, metrics={'train_runtime': 3040.4586, 'train_samples_per_second': 13.296, 'train_steps_per_second': 1.663, 'total_flos': 0.0, 'train_loss': 139655.40731948565, 'epoch': 15.0})

In [ ]:
trainer_base_ft.save_model("finetuned_gliner_model_base")#DOWNLOAD THIS MODEL

In [ ]:
best_ckpt_base_ft = trainer_base_ft.state.best_model_checkpoint
print(best_ckpt_base_ft)

gliner_model_base_ft/checkpoint-338


In [22]:
!zip -r finetuned_gliner_model_base.zip finetuned_gliner_model_base
from google.colab import files
files.download("finetuned_gliner_model_base.zip")

  adding: finetuned_gliner_model_base/ (stored 0%)
  adding: finetuned_gliner_model_base/added_tokens.json (deflated 36%)
  adding: finetuned_gliner_model_base/pytorch_model.bin (deflated 8%)
  adding: finetuned_gliner_model_base/special_tokens_map.json (deflated 50%)
  adding: finetuned_gliner_model_base/spm.model (deflated 50%)
  adding: finetuned_gliner_model_base/gliner_config.json (deflated 65%)
  adding: finetuned_gliner_model_base/tokenizer.json (deflated 77%)
  adding: finetuned_gliner_model_base/tokenizer_config.json (deflated 79%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [28]:
trained_model_base_ft = GLiNER.from_pretrained("finetuned_gliner_model_base", load_tokenizer=True)

config.json not found in /content/finetuned_gliner_model_base


In [24]:
with open("json_outputs.json", "r", encoding="utf-8") as file:
    json_outputs = json.load(file)

print(len(json_outputs))
print(json_outputs[0]['entities'][0]['types'])

2995
['body part']


In [25]:
json_outputs[0]

{'text': 'Any part of your neck  muscles, bones, joints, tendons, ligaments, or nerves  can cause neck problems. Neck pain is very common. Pain may also come from your shoulder, jaw, head, or upper arms. Muscle strain or tension often causes neck pain. The problem is usually overuse, such as from sitting at a computer for too long. Sometimes you can strain your neck muscles from sleeping in an awkward position or overdoing it during exercise. Falls or accidents, including car accidents, are another common cause of neck pain. Whiplash, a soft tissue injury to the neck, is also called neck sprain or strain. Treatment depends on the cause, but may include applying ice, taking pain relievers, getting physical therapy or wearing a cervical collar. You rarely need surgery.',
 'entities': [{'entity': 'neck', 'types': ['body part']},
  {'entity': 'muscles', 'types': ['body part']},
  {'entity': 'bones', 'types': ['body part']},
  {'entity': 'joints', 'types': ['body part']},
  {'entity': 'tend

In [30]:
entity_types_per_text = []
for i, output in enumerate(json_outputs):
  if i == 15:
    break
  entity_types = []
  for j, entity in enumerate(json_outputs[i]['entities']):
    for k, entity_type in enumerate(entity['types']):
      entity_types.append(entity_type)

  entity_types_per_text.append(
      {
          'text': output['text'],
          'entity_types': set(entity_types)
       }
      )

entity_types_per_text

[{'text': 'Any part of your neck  muscles, bones, joints, tendons, ligaments, or nerves  can cause neck problems. Neck pain is very common. Pain may also come from your shoulder, jaw, head, or upper arms. Muscle strain or tension often causes neck pain. The problem is usually overuse, such as from sitting at a computer for too long. Sometimes you can strain your neck muscles from sleeping in an awkward position or overdoing it during exercise. Falls or accidents, including car accidents, are another common cause of neck pain. Whiplash, a soft tissue injury to the neck, is also called neck sprain or strain. Treatment depends on the cause, but may include applying ice, taking pain relievers, getting physical therapy or wearing a cervical collar. You rarely need surgery.',
  'entity_types': {'body part', 'condition', 'event', 'treatment'}},
 {'text': "Heel problems are common and can be painful. Often, they result from too much stress on your heel bone and the tissues that surround it. Th

In [31]:
for item in entity_types_per_text:
  entities = trained_model_base_ft.predict_entities(item['text'], list(item['entity_types']), threshold=0.5)
  # print(item['text'])
  # print(item['entity_types'])
  for entity in entities:
    print(entity["text"], "=>", entity["label"])
  print("____________________________")

Any => body part
part => body part
of => body part
your => body part
neck => body part
muscles => body part
, => body part
bones => body part
, => body part
joints => body part
, => body part
tendons => body part
, => body part
ligaments => body part
, => body part
or => body part
nerves => body part
can => body part
cause => body part
neck => body part
problems => body part
. => body part
Neck => body part
pain => body part
is => body part
very => body part
common => body part
. => body part
Pain => body part
may => body part
also => body part
come => body part
from => body part
your => body part
shoulder => body part
, => body part
jaw => body part
, => body part
head => body part
, => body part
or => body part
upper => body part
arms => body part
. => body part
Muscle => body part
strain => body part
or => body part
tension => body part
often => body part
causes => body part
neck => body part
pain => body part
. => body part
The => body part
problem => body part
is => body part
usua